#Preparing Free Kaggle Dataset for Use in VAE

Kaggle Install

In [ ]:
# Source: https://www.kaggle.com/datasets/muratkokludataset/pistachio-image-dataset
!pip install -q kaggle

Upload Kaggle API token
* Run and then select token to upload
* To generate token go to Kaggle > Settings > Create New Token

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"heronziegel","key":"3dcf735973a043f1b1e838db2ee9923e"}'}

Create Directory and Move API Token There

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Download Dataset

In [5]:
!kaggle datasets download -d muratkokludataset/pistachio-image-dataset


Dataset URL: https://www.kaggle.com/datasets/muratkokludataset/pistachio-image-dataset
License(s): CC-BY-NC-SA-4.0
  0% 0.00/26.7M [00:00<?, ?B/s]
100% 26.7M/26.7M [00:00<00:00, 899MB/s]


Check If File Is There

In [7]:
!ls -lh /content/*.zip

-rw-r--r-- 1 root root 27M Mar 28  2022 /content/pistachio-image-dataset.zip


Mount Google Drive

In [13]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Move To New Folder

In [14]:
!mv /content/pistachio-image-dataset.zip /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios

Unzip Files

In [19]:
!unzip /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/pistachio-image-dataset.zip -d /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios

Archive:  /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/pistachio-image-dataset.zip
  inflating: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.arff  
  inflating: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.xls  
  inflating: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.xlsx  
  inflating: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset_Citation_Request.txt  
  inflating: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Pistachio_Image_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.arff  
  inflating: /content/dri

Resize All Images

In [20]:
import os
from PIL import Image
from pathlib import Path
from tqdm import tqdm

# Define source and destination paths
source_dir = '/content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/Kirmizi_Pistachio'  # Adjust to your actual folder
dest_dir = '/content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/256'

# Create destination directory
os.makedirs(dest_dir, exist_ok=True)

def resize_and_crop(image_path, output_path, target_size=(256, 256)):
    """
    Resize and center crop an image to target size
    """
    try:
        # Open image
        img = Image.open(image_path)

        # Convert to RGB if necessary (handles RGBA, grayscale, etc.)
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Calculate aspect ratios
        img_aspect = img.width / img.height
        target_aspect = target_size[0] / target_size[1]

        # Resize so that the smallest dimension matches target
        if img_aspect > target_aspect:
            # Image is wider, fit to height
            new_height = target_size[1]
            new_width = int(new_height * img_aspect)
        else:
            # Image is taller, fit to width
            new_width = target_size[0]
            new_height = int(new_width / img_aspect)

        img = img.resize((new_width, new_height), Image.LANCZOS)

        # Center crop to target size
        left = (new_width - target_size[0]) // 2
        top = (new_height - target_size[1]) // 2
        right = left + target_size[0]
        bottom = top + target_size[1]

        img = img.crop((left, top, right, bottom))

        # Save
        img.save(output_path, quality=95)
        return True

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return False

# Find all image files
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
image_files = []

for root, dirs, files in os.walk(source_dir):
    for file in files:
        if Path(file).suffix.lower() in image_extensions:
            image_files.append(os.path.join(root, file))

print(f"Found {len(image_files)} images to process")

# Process all images
successful = 0
failed = 0

for img_path in tqdm(image_files, desc="Processing images"):
    # Preserve folder structure
    relative_path = os.path.relpath(img_path, source_dir)
    output_path = os.path.join(dest_dir, relative_path)

    # Create subdirectories if needed
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Resize and crop
    if resize_and_crop(img_path, output_path):
        successful += 1
    else:
        failed += 1

print(f"\nProcessing complete!")
print(f"Successful: {successful}")
print(f"Failed: {failed}")
print(f"Output directory: {dest_dir}")

Found 1232 images to process


Processing images: 100%|██████████| 1232/1232 [00:39<00:00, 31.53it/s]


Processing complete!
Successful: 1232
Failed: 0
Output directory: /content/drive/MyDrive/Ethical_AI_Image/Images/Training/Pistachios/256
